# Preprocess the input files

The outputs of the IRIS simulator are available in the inputs folder

In [1]:
%ls -lah inputs

total 21M
drwxr-xr-x. 2 azonca oirlab  117 Oct 23 16:21 ./
drwxr-xr-x. 4 azonca oirlab  183 Oct 23 16:40 ../
-rw-r--r--. 1 azonca oirlab 5.4M Oct 23 16:20 raw_background_frame_cal.fits.gz
-rw-r--r--. 1 azonca oirlab 9.7M Oct 23 16:21 raw_flat_frame_cal.fits.gz
-rw-r--r--. 1 azonca oirlab 5.4M Oct 23 16:20 raw_science_frame_sci.fits.gz


Currently the JWST pipeline of course does not understand the IRIS keywords, we will implement them in the future, for now we just want to run the JWST pipeline as it is, therefore we strip all of the metadata.

In [2]:
from astropy.io import fits

In [3]:
science_file = fits.open("inputs/raw_science_frame_sci.fits.gz")

Then we want to use the convenient `ImageModel` class from the JWST pipeline which is a single object that contains the data (`.data`) array, data quality (`.dq`) array and error (`.err`) array.

In [4]:
from jwst.datamodels import ImageModel

In [5]:
im_science = ImageModel(data=science_file[0].data)

In [6]:
im_science.dq

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint32)

In [7]:
im_science.err

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

It also supports metadata and transparently serializes them to FITS headers when we write this data structure to disk. They are accessible inside the `.meta` attribute, currently we pretend that the data are from NIRCAM, in the future we will implement dedicated analysis modules just for IRIS and this won't be necessary.

In [8]:
# meta for bg subtraction
im_science.meta.instrument.name = "NIRCAM"
im_science.meta.exposure.type = "NRC_IMAGE"

In [9]:
# extra meta for flat
im_science.meta.instrument.detector = "NRCA1"

In [10]:
# flat-fielding also requires the subarray properties to be defined
im_science.meta.subarray.name = 'FULL'
im_science.meta.subarray.xsize = 2048
im_science.meta.subarray.xstart = 1
im_science.meta.subarray.ysize = 2048
im_science.meta.subarray.ystart = 1

In [11]:
im_science.write("sci.fits")

We can check what metadata was written to the FITS header

In [12]:
fits.open("sci.fits")[0].header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
                                                                                
        Level 3 Schema Metadata                                                 
                                                                                
DATE    = '2018-10-23T16:40:12.099' / [yyyy-mm-ddThh:mm:ss.ss] UTC date file cre
FILENAME= 'sci.fits'           / Name of the file                               
DATAMODL= 'ImageModel'         / Type of data model                             
                                                                                
        Instrument configuration information                                    
                            

## Preprocess the Background frame

We can repeat the same processing for the background frame, basically we just need to remove the IRIS headers which are not recognized by `stpipe` and write a few JWST metadata:

In [13]:
im_bg = ImageModel(data=fits.open("inputs/raw_background_frame_cal.fits.gz")[0].data)
im_bg.meta.instrument.name = "NIRCAM"
im_bg.meta.exposure.type = "NRC_IMAGE"

In [14]:
im_bg.write("bg.fits")

## Preprocess the Flat frame

The JWST flat fielding module assumes that the Flat frame is already background subtracted and calibrated, therefore we need to do it beforehand

In [15]:
flat = fits.open("inputs/raw_flat_frame_cal.fits.gz")

In [16]:
im_flat =  ImageModel(data=flat[0].data)

In [17]:
im_flat.meta.instrument.name = "NIRCAM"
im_flat.meta.exposure.type = "NRC_IMAGE"

In [18]:
im_flat.data -= im_bg.data
im_flat.data /= im_flat.data.max()

In [19]:
im_flat.write("flat.fits")